In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from sklearn.datasets import load_breast_cancer 
from sklearn import decomposition, datasets
from sklearn.svm import SVC, SVR
import os, sys
sys.path.append(r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\default')
sys.path.append(r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\default\External_Files')
#custom functions
import my_functions
from cf_matrix import make_confusion_matrix

In [2]:
workdir = r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\HW4_SV'
data_dir = r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\default\DataSets'
graph_dir = r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\HW4_SV\HW4_Graphs'
HW4_data = r'C:\Users\noaht\Documents\Intro To ML\INTRO_TO_ML_REPO\HW4_SV\HW4_Data'

In [3]:
############################  ALGORITHM  ###########################################
# Prepare data for problems 1 and 2 

# Write Function That Preforms SVM Model 
# Write Function that preforms SVR Model
# Write function (HW4Function()) for Identifing The optiinal K for PCA then runs 
# the model through two different Kernels. 

# Problem 1: 
# HW4Function() 

# Prooblem 2: 
# Run SVR Model on the data
# HW4Function() 
####################################################################################

# Clense the Data (P1&P2)

In [4]:
# Import the Cancer dataset 
df_c = pd.read_csv(data_dir + '/cancer.csv')
df_c = df_c.astype({'label': 'int32'})

In [5]:
# Import the Housing Dataset 
df_h = pd.read_csv(data_dir + '/Housing.csv')
# Select the desired inputs
desired_inputs = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'price']
df_h = df_h[desired_inputs]
df_h = my_functions.cat_to_useful(df_h)

# Functions 

In [6]:
# SplitData(df, Y_Name, split= .2, rand_state)
# INPUTS: df (a dataframe), Y_name (name of Y column), split (test to train ratio), rand_state
# OUTPUTS: X_test, Y_test, X_train, Y_train
def SplitData(df, Y_name, split= .2, rand_state = 0):
    X = df.loc[:, df.columns != Y_name ]
    Y = df.loc[:, df.columns == Y_name]
    return train_test_split(X, Y, test_size=split, random_state=rand_state)

In [7]:
# SVClass() Function 
# INPUTS:  X_train, Y_train, X_test, kernel= 'linear' , C= 10 
# OUTPUTS:accuracy, recall, precision
def SVClass( df, Y_name, kernel= 'linear', C= 100):
    X_train, X_test, Y_train, Y_test = SplitData(df, Y_name)
    model = SVC(kernel= kernel, C=C)
    Y_pred = model.fit(X_train, Y_train).predict(X_test)
    return my_functions.class_report(Y_test, Y_pred)


In [8]:
# SVReg() Function
# INPUTS: X_train, Y_train, X_test, kernel= 'linear' , C= 10
# OUTPUTS: r2 accuracy
def SVReg(df, Y_name, kernel= 'linear', C= 100):
    X_train, X_test , Y_train, Y_test = SplitData(df, Y_name)
    model = SVR(kernel=kernel, C=C)
    Y_pred = model.fit(X_train, Y_train).predict(X_test)
    return metrics.r2_score(Y_test, Y_pred)



In [9]:
# Graph Function 
# Requried Imports: matplotlib 
# INPUTS: DataFrame, column index, Title, Path RETURNS: Nothing
def plot_DataFrame(df, x_column, Title= 'Default', path= None):
    columns = list(df.columns)
    columns.remove(x_column)
    for column in columns:
        plt.plot(df[x_column],df[column], label = column)
    plt.title(Title)
    plt.grid()
    plt.xlabel('K Value')
    plt.legend()
    if path:
        plt.savefig(path + '/' + Title + '.jpg')
    plt.clf()

In [10]:
################################## ALGORITHM ################################
# 1. Stanardize the data
# 2. Use PCA on the data to figure out the best accuracy
# 3. Plot The PCA itterations 
# 4. Use the best K to Run PCA again and Run different kernels on the model
# 5. Return the accuracys and plot if its a regression.

def HW4Function(df, problem, Y_name, kernel):
    # Create Nessasary Variables IE dataframe that holds the k values 
    
     # K value, Accuracy, Recall, Percsision
    if problem == 'problem 1':
        template = {'K': [], 'accuracy': [], 'recall': [], 'percision': []}
        classification = Y_name
        func = SVClass
    else: 
        template = {'K': [], 'accuracy': []}
        classification = None 
        func = SVReg
    temp_k = [0]
    temp = [0]
    results = pd.DataFrame(template)
    df = my_functions.clense_data(df, stand= True, classification= classification)
    temp_df = df.copy()
    for i in range(len(df.columns)-1):
        temp_k[0] = i + 1 
        temp_df = my_functions.DemReduction(temp_df, i + 1, Y_name)
        if problem == 'problem 1':
            temp = temp_k + list(func(temp_df,Y_name, kernel= kernel))  
            results.loc[len(results)] = temp 
        else: 
            temp_k.append(func(temp_df,Y_name, kernel= kernel)) 
            temp = temp_k
            results.loc[len(results)] = temp
            del temp_k[-1]
        temp_df = df
    plot_DataFrame(results, 'K', Title = str(problem) + '_' +str(kernel), path = graph_dir)
    # find the K value with the best results 
    return list(results.loc[results.idxmax(0)['accuracy']])

    
      
        
    

# Problem 1

In [11]:
# Declare Variables 
kernels_used = ['linear', 'rbf', 'poly']
template = {'K': [], 'accuracy': [], 'recall': [], 'percision': []}
results_df = pd.DataFrame(template)
temp_df = pd.DataFrame({'Kernal Type': kernels_used})

for kernel in kernels_used: 
    results_df.loc[len(results_df)]= HW4Function(df_c, 'problem 1', 'label', kernel)
my_functions.create_data_table([pd.concat([temp_df,results_df], axis = 0)], spreadsheet_name= 'problem 1', path = HW4_data)

        


c:\Users\noaht\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWa

<Figure size 432x288 with 0 Axes>

# Problem 2

In [12]:
# Declare Variables 
kernels_used = ['linear', 'rbf', 'poly']
template = {'K': [], 'accuracy': []}
results_df = pd.DataFrame(template)
temp_df = pd.DataFrame({'Kernal Type': kernels_used})

for kernel in kernels_used: 
    print(HW4Function(df_h, 'problem 2', 'price', kernel))
    results_df.loc[len(results_df)]= HW4Function(df_h, 'problem 2', 'price', kernel)
my_functions.create_data_table([pd.concat([temp_df,results_df], axis = 0)],spreadsheet_name= 'problem 2', path = HW4_data)


c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

[11.0, 0.6712634456366584]


c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

[2.0, 0.6241813537123054]


c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

[4.0, 0.3314197433953625]


c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\Users\noaht\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

<Figure size 432x288 with 0 Axes>

In [13]:
temp = [0]
temp[0] = 1
temp.append(3.3)
del temp[-1]
print(temp)

[1]
